# Notebook 2: Important Nodes Detection

This notebook processes the primary graph **G1** to detect structurally important nodes using **K-core importance** and **Betweenness Centrality**.  

The results are saved for future augmentation.


In [1]:
import os
dir_path = os.getcwd()
print("The directory of this script is:", dir_path)
root_path = os.path.dirname(dir_path)
print("The root directory is:", root_path)

The directory of this script is: c:\Users\HP\Desktop\Projects\NodeRAG\graphs
The root directory is: c:\Users\HP\Desktop\Projects\NodeRAG


In [2]:
#import Node class definition
import sys
sys.path.append(root_path)
from graphs.Node import Node

In [3]:
#load primary graph G1 with S,N,R nodes
import pickle
with open(f"{root_path}/graphs/data/graphs/G1_medical_primary_graph.pkl", "rb") as f:
    medical_nodes = pickle.load(f)

In [4]:
#find k-core important nodes: entity nodes with number of connection >= k_default
import math
def k_default(nodes):
    V = len(nodes)
    sum_deg = sum([node.degree for node in nodes.values()])
    k_avg = sum_deg / V
    k_def = math.floor(math.log(V) * math.sqrt(k_avg))
    return k_def

def k_core_importance(nodes):
    k = k_default(nodes)
    important_nodes = {node_id: node for node_id, node in nodes.items() if node.degree >= k and node.node_type == "N"}
    return important_nodes

k_core_nodes_medical = k_core_importance(medical_nodes)
len(k_core_nodes_medical)

589

In [5]:
import random
import heapq
from tqdm import tqdm

def approximate_betweenness_weighted(nodes_dict, k_sample=10):
    #Betweenness centrality measures how frequently a node appears on the shortest path between a pair of nodes in the graph.
    #Brandes' algorithm with sampling (approximate using k nodes instead of all nodes)
    node_ids = list(nodes_dict.keys())
    sampled_nodes = random.sample(node_ids, min(k_sample, len(node_ids)))  # pick k nodes randomly
    bc = {node_id: 0.0 for node_id in node_ids}  #initialize
    
    for i in tqdm(range(k_sample)):
        s_id = sampled_nodes[i]
        #Dijkstra shortest paths from s_id
        dist = {nid: float('inf') for nid in node_ids}
        num_paths = {nid: 0 for nid in node_ids}
        pred = {nid: [] for nid in node_ids}
        #init
        dist[s_id] = 0
        num_paths[s_id] = 1
        heap = [(0, s_id)]
        
        while heap:
            d_u, u = heapq.heappop(heap)
            if d_u > dist[u]:
                continue
            
            for v, w in nodes_dict[u].edges.items():
                alt = dist[u] + w
                if alt < dist[v]:
                    dist[v] = alt
                    heapq.heappush(heap, (alt, v))
                    num_paths[v] = num_paths[u]
                    pred[v] = [u]
                elif alt == dist[v]:
                    num_paths[v] += num_paths[u]
                    pred[v].append(u)
        
        #backpropagation: accumulate dependencies - how much each node lies on top of the shortest path between the source node and other nodes
        delta = {nid: 0 for nid in node_ids} #dependency scores
        nodes_by_dist = sorted(dist.items(), key=lambda x: -x[1]) #decreasing order of distance: dependency propagates from the leaf nodes up to the source
        for v, _ in nodes_by_dist:
            for u in pred[v]:
                # fraction of shortest paths through u
                if num_paths[v] > 0:
                    delta[u] += (num_paths[u] / num_paths[v]) * (1 + delta[v])
            if v != s_id:
                bc[v] += delta[v]
    
    #threshold
    for v in bc:
        bc[v] /= k_sample
    avg_bc = sum(bc.values()) / len(bc)
    scale = int(math.log10(len(nodes_dict)))
    important_nodes = {v: nodes_dict[v] for v, score in bc.items() if score > avg_bc * scale and nodes_dict[v].node_type == "N"}
    
    return important_nodes

bc_medical_nodes = approximate_betweenness_weighted(medical_nodes, k_sample=len(medical_nodes)//10)

print("Number of important medical nodes:", len(bc_medical_nodes))


100%|██████████| 3144/3144 [11:19<00:00,  4.62it/s]

Number of important medical nodes: 555


In [6]:
#sanity check
def shared_keys_different_values(dict1, dict2):
    for key in dict1:
        if key in dict2 and dict1[key] != dict2[key]:
            return True
    return False

print(shared_keys_different_values(k_core_nodes_medical, bc_medical_nodes))

False


In [7]:
#union of 2 types of important nodes
important_medical_nodes = {**k_core_nodes_medical, **bc_medical_nodes}
print("Total important medical nodes:", len(important_medical_nodes))

Total important medical nodes: 686


In [8]:
import json

with open("data/nodes/attribute/important_medical_nodes.json", "w") as f:
    json.dump(list(important_medical_nodes.keys()), f, indent=2)
with open("data/nodes/attribute/important_medical_nodes.json", "r") as f:
    important_medical_nodes_loaded = json.load(f)

In [9]:
#sanity check
for id in important_medical_nodes_loaded:
    if id not in medical_nodes:
        print("Invalid Medical ID:", id)
print("-"*100)

----------------------------------------------------------------------------------------------------
